First import the modules we need:

In [1]:
import trajectory, trajectory.post, trajectory.aero, csv
import numpy as np

To run an aerodynamic simulation, we must have already run a trajectory simulation and saved the output to a .JSON. We will now import that JSON:

In [2]:
imported_data = trajectory.from_json("output.json")

We need to recreate the Rocket object that was used to generate output.json. This is because output.json only contains trajectory data, but no details on the actual launch vehicle.

In [3]:
'''Import motor data - copied from Joe Hunt's simulation'''
with open('novus_sim_6.1/motor_out.csv') as csvfile:
    motor_out = csv.reader(csvfile)

    (motor_time_data, prop_mass_data, cham_pres_data,
     throat_data, gamma_data, nozzle_efficiency_data,
     exit_pres_data, area_ratio_data, vmass_data, lden_data, lmass_data, fuel_mass_data) = [], [], [], [], [], [], [], [], [], [], [], []

    next(motor_out)
    for row in motor_out:
        motor_time_data.append(float(row[0]))
        prop_mass_data.append(float(row[1]))
        cham_pres_data.append(float(row[2]))
        throat_data.append(float(row[3]))
        gamma_data.append(float(row[4]))
        nozzle_efficiency_data.append(float(row[5]))
        exit_pres_data.append(float(row[6]))
        area_ratio_data.append(float(row[7]))
        vmass_data.append(float(row[8]))
        lden_data.append(float(row[9]))
        lmass_data.append(float(row[10]))
        fuel_mass_data.append(float(row[11]))
        
        #This is a bit inefficient given that these are constants, (we only need to record them once):
        DENSITY_FUEL = float(row[12])
        DIA_FUEL = float(row[13])
        LENGTH_PORT = float(row[14])

'''Rocket parameters'''
dry_mass = 60                               # kg
rocket_length = 6.529                       # m
rocket_radius = 98.5e-3                     # m
rocket_wall_thickness = 1e-2                # m - This is just needed for the mass model
pos_tank_bottom = 4.456                     # m - Distance between the nose tip and the bottom of the nitrous tank
pos_solidfuel_bottom = 4.856+LENGTH_PORT    # m - Distance between the nose tip and bottom of the solid fuel grain 
ref_area = 0.0305128422                     # m^2 - Reference area for aerodynamic coefficients

'''Set up aerodynamic properties'''
#Get approximate values for the rotational damping coefficients
c_damp_pitch = trajectory.aero.pitch_damping_coefficient(rocket_length, rocket_radius, fin_number = 4, area_per_fin = 0.07369928)
c_damp_roll = 0

#Import drag coefficients from RASAero II
aerodynamic_coefficients = trajectory.aero.RASAeroData("data/Martlet4RasAeroII.CSV", ref_area, c_damp_pitch, c_damp_roll)

'''Set up the mass model'''
liquid_fuel = trajectory.LiquidFuel(lden_data, lmass_data, rocket_radius, pos_tank_bottom, motor_time_data)
solid_fuel = trajectory.SolidFuel(fuel_mass_data, DENSITY_FUEL, DIA_FUEL/2, LENGTH_PORT, pos_solidfuel_bottom, motor_time_data)
dry_mass_model = trajectory.HollowCylinder(rocket_radius, rocket_radius - rocket_wall_thickness, rocket_length, dry_mass)

mass_model = trajectory.HybridMassModel(rocket_length, solid_fuel, liquid_fuel, vmass_data, 
                                        dry_mass_model.mass, dry_mass_model.ixx(), dry_mass_model.iyy(), dry_mass_model.izz(), 
                                        dry_cog = rocket_length/2)

'''Create the other objects needed to initialise the Rocket object'''
pulsar = trajectory.Motor(motor_time_data, 
                          prop_mass_data, 
                          cham_pres_data, 
                          throat_data, 
                          gamma_data, 
                          nozzle_efficiency_data, 
                          exit_pres_data, 
                          area_ratio_data)

launch_site = trajectory.LaunchSite(rail_length=10, 
                                    rail_yaw=0, 
                                    rail_pitch=0, 
                                    alt=0, 
                                    longi=0, 
                                    lat=0, 
                                    wind=[4.94975,4.94975,0])

parachute = trajectory.Parachute(main_s = 13.9,
                                 main_c_d = 0.78,
                                 drogue_s = 1.13,
                                 drogue_c_d = 0.78,
                                 main_alt = 1000,
                                 attatch_distance = 0)

"""Create the Rocket object"""
martlet4 = trajectory.Rocket(mass_model, pulsar, aerodynamic_coefficients, launch_site, h=0.05, variable=True, alt_poll_interval=1, parachute=parachute)


Now we can start actually setting up an aerodynamic heating simulation. First we must define the nosecone shape, by creating a TangentOgive object:

In [4]:
tangent_ogive = trajectory.post.TangentOgive(xprime = 73.7e-2, yprime = (19.7e-2)/2)

'xprime' is the length of the nosecone, and 'yprime' is the radius. See Fig 2. of https://arc.aiaa.org/doi/pdf/10.2514/3.62081 for a diagram.

We can now set up the aerodynamic heating simulation, and run a step:

In [ ]:
analysis = trajectory.post.HeatTransfer(tangent_ogive, imported_data, martlet4)
analysis.step()

Subsonic flow, skipping step number 0


In [ ]:
analysis.run(iterations = 300)

Subsonic flow, skipping step number 1
Subsonic flow, skipping step number 2
Subsonic flow, skipping step number 3
Subsonic flow, skipping step number 4
Subsonic flow, skipping step number 5
Subsonic flow, skipping step number 6
Subsonic flow, skipping step number 7
Subsonic flow, skipping step number 8
Subsonic flow, skipping step number 9
Subsonic flow, skipping step number 10
Subsonic flow, skipping step number 11
Subsonic flow, skipping step number 12
Subsonic flow, skipping step number 13
Subsonic flow, skipping step number 14
Subsonic flow, skipping step number 15
Subsonic flow, skipping step number 16
Subsonic flow, skipping step number 17
Subsonic flow, skipping step number 18
Subsonic flow, skipping step number 19
Subsonic flow, skipping step number 20
Subsonic flow, skipping step number 21
Subsonic flow, skipping step number 22
Subsonic flow, skipping step number 23
Subsonic flow, skipping step number 24
Subsonic flow, skipping step number 25
Subsonic flow, skipping step numbe

 D:\Anaconda3\lib\site-packages\thermo\viscosity.py:826: RuntimeWarning:invalid value encountered in double_scalars
 D:\OneDrive - University Of Cambridge\Cambridge\CUSF\6DOF-Trajectory-Simulation\trajectory\post.py:526: RuntimeWarning:divide by zero encountered in double_scalars


i=100 station=1 q_lam = 0.0 W/m^2 alt = 1371.6184645770118 m
i=100 station=2 q_lam = 238.8630844329018 W/m^2 alt = 1371.6184645770118 m
i=100 station=3 q_lam = 451.00718951017626 W/m^2 alt = 1371.6184645770118 m
i=100 station=4 q_lam = 638.3644680047059 W/m^2 alt = 1371.6184645770118 m
i=100 station=5 q_lam = 797.3791958325075 W/m^2 alt = 1371.6184645770118 m
i=100 station=6 q_lam = 927.3451163853784 W/m^2 alt = 1371.6184645770118 m
i=100 station=7 q_lam = 1028.925181760718 W/m^2 alt = 1371.6184645770118 m
i=100 station=8 q_lam = 1103.5655991179885 W/m^2 alt = 1371.6184645770118 m
i=100 station=9 q_lam = 1153.198205992983 W/m^2 alt = 1371.6184645770118 m
i=100 station=10 q_lam = 1180.0609169179886 W/m^2 alt = 1371.6184645770118 m
i=100 station=11 q_lam = 1592.132909532788 W/m^2 alt = 1371.6184645770118 m
i=100 station=12 q_lam = 1813.7283090083324 W/m^2 alt = 1371.6184645770118 m
i=100 station=13 q_lam = 1956.7105314964317 W/m^2 alt = 1371.6184645770118 m
i=100 station=14 q_lam = 2062.

i=127 station=5 q_lam = 2345.519810015607 W/m^2 alt = 4547.209642529488 m
i=127 station=6 q_lam = 2775.4123103412494 W/m^2 alt = 4547.209642529488 m
i=127 station=7 q_lam = 3132.6361720162736 W/m^2 alt = 4547.209642529488 m
i=127 station=8 q_lam = 3418.9177376117914 W/m^2 alt = 4547.209642529488 m
i=127 station=9 q_lam = 3637.736893198098 W/m^2 alt = 4547.209642529488 m
i=127 station=10 q_lam = 3793.7169025606945 W/m^2 alt = 4547.209642529488 m
i=127 station=11 q_lam = 4317.442558014206 W/m^2 alt = 4547.209642529488 m
i=127 station=12 q_lam = 4687.745033155472 W/m^2 alt = 4547.209642529488 m
i=127 station=13 q_lam = 4981.593589237701 W/m^2 alt = 4547.209642529488 m
i=127 station=14 q_lam = 5230.824460259467 W/m^2 alt = 4547.209642529488 m
i=127 station=15 q_lam = 5469.625534355178 W/m^2 alt = 4547.209642529488 m
i=128 station=1 q_lam = 0.0 W/m^2 alt = 4729.751877190545 m
i=128 station=2 q_lam = 678.8328938023366 W/m^2 alt = 4729.751877190545 m
i=128 station=3 q_lam = 1309.9423600457867

i=134 station=11 q_lam = 4839.125497035598 W/m^2 alt = 5426.520012905821 m
i=134 station=12 q_lam = 5257.359698082826 W/m^2 alt = 5426.520012905821 m
i=134 station=13 q_lam = 5592.979558893771 W/m^2 alt = 5426.520012905821 m
i=134 station=14 q_lam = 5879.63827024551 W/m^2 alt = 5426.520012905821 m
i=134 station=15 q_lam = 6154.347008529759 W/m^2 alt = 5426.520012905821 m
i=135 station=1 q_lam = 0.0 W/m^2 alt = 5435.75750639569 m
i=135 station=2 q_lam = 743.0346601578889 W/m^2 alt = 5435.75750639569 m
i=135 station=3 q_lam = 1433.7707515858679 W/m^2 alt = 5435.75750639569 m
i=135 station=4 q_lam = 2070.5123739427218 W/m^2 alt = 5435.75750639569 m
i=135 station=5 q_lam = 2633.784283370749 W/m^2 alt = 5435.75750639569 m
i=135 station=6 q_lam = 3116.8180204829227 W/m^2 alt = 5435.75750639569 m
i=135 station=7 q_lam = 3518.6738489021127 W/m^2 alt = 5435.75750639569 m
i=135 station=8 q_lam = 3841.622384078181 W/m^2 alt = 5435.75750639569 m
i=135 station=9 q_lam = 4089.8856959184327 W/m^2 alt

...

i=172 station=10 q_lam = 3533.2658593704837 W/m^2 alt = 6680.158038491383 m
i=172 station=11 q_lam = 4013.935213942572 W/m^2 alt = 6680.158038491383 m
i=172 station=12 q_lam = 4360.342913852708 W/m^2 alt = 6680.158038491383 m
i=172 station=13 q_lam = 4638.336418323565 W/m^2 alt = 6680.158038491383 m
i=172 station=14 q_lam = 4875.770453394674 W/m^2 alt = 6680.158038491383 m
i=172 station=15 q_lam = 5103.35057007849 W/m^2 alt = 6680.158038491383 m
i=173 station=1 q_lam = 0.0 W/m^2 alt = 6705.389990004711 m
i=173 station=2 q_lam = 614.499018209422 W/m^2 alt = 6705.389990004711 m
i=173 station=3 q_lam = 1185.1581322872678 W/m^2 alt = 6705.389990004711 m
i=173 station=4 q_lam = 1710.636811232748 W/m^2 alt = 6705.389990004711 m
i=173 station=5 q_lam = 2174.913035319923 W/m^2 alt = 6705.389990004711 m
i=173 station=6 q_lam = 2572.4877860622496 W/m^2 alt = 6705.389990004711 m
i=173 station=7 q_lam = 2902.683508021449 W/m^2 alt = 6705.389990004711 m
i=173 station=8 q_lam = 3167.4726631774183 W/

i=180 station=1 q_lam = 0.0 W/m^2 alt = 7087.1419593747705 m
i=180 station=2 q_lam = 572.2637313351265 W/m^2 alt = 7087.1419593747705 m
i=180 station=3 q_lam = 1103.4200642314543 W/m^2 alt = 7087.1419593747705 m
i=180 station=4 q_lam = 1592.2387329763671 W/m^2 alt = 7087.1419593747705 m
i=180 station=5 q_lam = 2023.8645739537878 W/m^2 alt = 7087.1419593747705 m
i=180 station=6 q_lam = 2393.2224937914452 W/m^2 alt = 7087.1419593747705 m
i=180 station=7 q_lam = 2699.718667186449 W/m^2 alt = 7087.1419593747705 m
i=180 station=8 q_lam = 2945.221762487343 W/m^2 alt = 7087.1419593747705 m
i=180 station=9 q_lam = 3133.0915523378844 W/m^2 alt = 7087.1419593747705 m
i=180 station=10 q_lam = 3267.6302736673056 W/m^2 alt = 7087.1419593747705 m
i=180 station=11 q_lam = 3713.0159987126 W/m^2 alt = 7087.1419593747705 m
i=180 station=12 q_lam = 4033.595796551201 W/m^2 alt = 7087.1419593747705 m
i=180 station=13 q_lam = 4290.623522355474 W/m^2 alt = 7087.1419593747705 m
i=180 station=14 q_lam = 4510.0

i=187 station=13 q_lam = 4098.497882845252 W/m^2 alt = 7294.785579290241 m
i=187 station=14 q_lam = 4308.952682829056 W/m^2 alt = 7294.785579290241 m
i=187 station=15 q_lam = 4510.726206433742 W/m^2 alt = 7294.785579290241 m
i=188 station=1 q_lam = 0.0 W/m^2 alt = 7362.826345349662 m
i=188 station=2 q_lam = 537.2356175439966 W/m^2 alt = 7362.826345349662 m
i=188 station=3 q_lam = 1035.7106532058824 W/m^2 alt = 7362.826345349662 m
i=188 station=4 q_lam = 1494.2814328711304 W/m^2 alt = 7362.826345349662 m
i=188 station=5 q_lam = 1899.022930599681 W/m^2 alt = 7362.826345349662 m
i=188 station=6 q_lam = 2245.1966690720315 W/m^2 alt = 7362.826345349662 m
i=188 station=7 q_lam = 2532.2723653879693 W/m^2 alt = 7362.826345349662 m
i=188 station=8 q_lam = 2762.030829486692 W/m^2 alt = 7362.826345349662 m
i=188 station=9 q_lam = 2937.6509264466863 W/m^2 alt = 7362.826345349662 m
i=188 station=10 q_lam = 3063.1938166061996 W/m^2 alt = 7362.826345349662 m
i=188 station=11 q_lam = 3489.067514544782

i=195 station=10 q_lam = 2805.659671926919 W/m^2 alt = 7747.598042072728 m
i=195 station=11 q_lam = 3194.3899470244123 W/m^2 alt = 7747.598042072728 m
i=195 station=12 q_lam = 3472.8318090753505 W/m^2 alt = 7747.598042072728 m
i=195 station=13 q_lam = 3695.2199390941228 W/m^2 alt = 7747.598042072728 m
i=195 station=14 q_lam = 3884.531682856987 W/m^2 alt = 7747.598042072728 m
i=195 station=15 q_lam = 4066.0600470721693 W/m^2 alt = 7747.598042072728 m
i=196 station=1 q_lam = 0.0 W/m^2 alt = 7771.018203174695 m
i=196 station=2 q_lam = 489.70334520967924 W/m^2 alt = 7771.018203174695 m
i=196 station=3 q_lam = 943.9056014826278 W/m^2 alt = 7771.018203174695 m
i=196 station=4 q_lam = 1361.5414975799579 W/m^2 alt = 7771.018203174695 m
i=196 station=5 q_lam = 1729.961245073079 W/m^2 alt = 7771.018203174695 m
i=196 station=6 q_lam = 2044.8628394797877 W/m^2 alt = 7771.018203174695 m
i=196 station=7 q_lam = 2305.772129512914 W/m^2 alt = 7771.018203174695 m
i=196 station=8 q_lam = 2514.3193440394

i=203 station=1 q_lam = 0.0 W/m^2 alt = 8015.741079584695 m
i=203 station=2 q_lam = 460.77025903579994 W/m^2 alt = 8015.741079584695 m
i=203 station=3 q_lam = 887.8676778899958 W/m^2 alt = 8015.741079584695 m
i=203 station=4 q_lam = 1280.3115223525779 W/m^2 alt = 8015.741079584695 m
i=203 station=5 q_lam = 1626.2615126602093 W/m^2 alt = 8015.741079584695 m
i=203 station=6 q_lam = 1921.7151746550148 W/m^2 alt = 8015.741079584695 m
i=203 station=7 q_lam = 2166.263785991576 W/m^2 alt = 8015.741079584695 m
i=203 station=8 q_lam = 2361.4734933761306 W/m^2 alt = 8015.741079584695 m
i=203 station=9 q_lam = 2510.105170606342 W/m^2 alt = 8015.741079584695 m
i=203 station=10 q_lam = 2615.673069629933 W/m^2 alt = 8015.741079584695 m
i=203 station=11 q_lam = 2981.9965350347534 W/m^2 alt = 8015.741079584695 m
i=203 station=12 q_lam = 3243.624023253959 W/m^2 alt = 8015.741079584695 m
i=203 station=13 q_lam = 3452.0918262176388 W/m^2 alt = 8015.741079584695 m
i=203 station=14 q_lam = 3629.25965340997